In [1]:
import sys

# Add the parent directory to sys.path
sys.path.append("/home/jovyan/video-storage/amit_files/")

In [2]:
import os   
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import imageio
from tqdm import trange
import tensorflow.compat.v1 as tf
from torch.utils.data import DataLoader, Dataset
import gc
import pickle
import yaml
from torch import nn

2025-04-25 16:31:55.002033: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-25 16:31:56.196963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.sync_batchnorm import DataParallelWithCallback
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.augmentation import SelectRandomFrames, SelectFirstFrames_two, VideoToTensor
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.frames_dataset import FramesDataset
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.PREDICTOR.Source_Model.prediction_toplevel import KPDataset,get_data_from_dataloader_60
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.generator import OcclusionAwareGenerator,calculate_frechet_distance,compute_fvd
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.keypoint_detector import KPDetector
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer, Visualizer_slow

from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.keypoint_detector import ModifiedKPDetector, KPDetector
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.FOMM.Source_Model.modules.RNN_prediction_module import PredictionModule
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.PREDICTOR.Source_Model.VRNN import build_vrnn, get_config
from Motion_Transfer_Keypoints_Prediction.Keypoints_Prediction.Training_Prediction.PREDICTOR.Source_Model.VRNN_prediction import VRNN_predict



import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]='0'

# Load the pretrained VRNN Model

In [4]:
model = build_vrnn
frames = 24
cfg = get_config()
input_keypoint = tf.keras.Input(shape=[frames,10,6],name='keypoint')
observed_keypoints_stop = tf.keras.layers.Lambda(tf.stop_gradient)(
input_keypoint)
vrnn_model = model(cfg)
predicted_keypoints, kl_divergence = vrnn_model(observed_keypoints_stop)
train_model = tf.keras.Model(inputs=[input_keypoint],outputs=[predicted_keypoints])
vrnn_coord_pred_loss = tf.nn.l2_loss(
observed_keypoints_stop - predicted_keypoints)
# Normalize by batch size and sequence length:
vrnn_coord_pred_loss /= tf.to_float(
  tf.shape(input_keypoint)[0] * tf.shape(input_keypoint)[1])
train_model.add_loss(vrnn_coord_pred_loss)
kl_loss = tf.reduce_mean(kl_divergence)  # Mean over batch and timesteps.
train_model.add_loss(cfg.kl_loss_scale * kl_loss)

# Load saved model:
cfg = get_config()
checkpoint_path = "/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/Checkpoints/VRNN_3883videos_vox_12-12.ckpt" 
# checkpoint_path = "Checkpoints/VRNN_3883videos_vox_6-6.ckpt" 

# Loads the weights
train_model.load_weights(checkpoint_path)
train_model.reset_states()

2025-04-25 16:32:03.745248: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10437 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:44:00.0, compute capability: 6.1


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use `tf.cast` instead.


# Import keypoints of 44 VoxCeleb test videos

In [5]:
with open("/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/kp_test_44_vox.pkl", "rb") as f:
    kp_time_series = pickle.load(f)
len(kp_time_series)

44

# Convert list of keypoints to dictionary

In [6]:
for video_idx in range(len(kp_time_series)):
    kp_time_series[video_idx] = kp_time_series[video_idx]['kp']

kp_dict_init = []
for video_idx in range(len(kp_time_series)): # 
    init_mean = []
    init_jacobian = []
    for frame_idx in range(len(kp_time_series[video_idx])):
        kp_mean = kp_time_series[video_idx][frame_idx]['value'].reshape(1,10,2)
        kp_mean = torch.tensor(kp_mean)
        kp_jacobian = kp_time_series[video_idx][frame_idx]['jacobian'].reshape(1,10,2,2)
        kp_jacobian = torch.tensor(kp_jacobian)

        init_mean.append(kp_mean)
        init_jacobian.append(kp_jacobian)

    init_mean = torch.cat(init_mean)
    init_jacobian = torch.cat(init_jacobian)

    init_mean = torch.reshape(init_mean,(1,init_mean.shape[0],init_mean.shape[1],init_mean.shape[2]))
    init_jacobian = torch.reshape(init_jacobian,(1,init_jacobian.shape[0],10,2,2))

    if torch.cuda.is_available():
        # add tensor to cuda
        init_mean = init_mean.to('cuda:0')
        init_jacobian = init_jacobian.to('cuda:0')

    kp_dict_both = {"value":init_mean,"jacobian":init_jacobian}
    kp_dict_init.append(kp_dict_both)

# Apply min-max std to keypoints and convert to batches

In [7]:
kp_list_test = []
for video_idx in range(len(kp_dict_init)):
    kp_one_video = torch.cat((kp_dict_init[video_idx]['value'], kp_dict_init[video_idx]['jacobian'].reshape(1,-1,10,4)),dim=-1).reshape(-1,60)
    kp_one_video_array = np.array(kp_one_video.cpu())
    kp_list_test.append(kp_one_video_array)
    
#####  min-max std to 60 dimensions of selected one video ######
kp_list_test_std = []
min_list = []
range_list = []
for video_idx in range(len(kp_list_test)):
    data = kp_list_test[video_idx]
    data_length = len(kp_list_test[video_idx])
    step_interval = 12 # choose between 12 frames or 24 frames 
    min_required_steps = 2*step_interval
    selected_data = []
    for i in range(0, data_length - min_required_steps+1, 2 * step_interval):
        selected_data.extend(data[i:i + step_interval])
    min_values = np.min(selected_data,axis=0) # 60 mins of one selected video in the loop
    max_values = np.max(selected_data,axis=0) # 60 maxs of one selected video in the loop 
    range_values = max_values - min_values 
    kp_one_video_std = (kp_list_test[video_idx] - min_values) / range_values
    kp_list_test_std.append(kp_one_video_std)
    min_list.append(min_values)
    range_list.append(range_values)

trajs = kp_list_test_std
print(len(trajs))
print(trajs[0].shape)

44
(118, 60)


In [8]:
######### convert into batches:
frames = min_required_steps
input_frames = int(frames / 2)
data_batch_test = []
for t,x in enumerate(kp_list_test_std):
    if x.shape[0] >= frames:
        num_full_batches = x.shape[0] // frames
        for arr in np.array_split(x[:num_full_batches * frames], num_full_batches):
            data_batch_test.append(arr)
print(f'test dataset batches:', len(data_batch_test))
print(data_batch_test[0].shape)

test dataset batches: 529
(24, 60)


In [9]:
###### test dataset:

test_data_reshape = np.array(data_batch_test).reshape(-1,frames,60)
test_data_reshape.shape

(529, 24, 60)

# Predict keypoints using trained model:

In [10]:
# Load test dataset and process model.predict():

validation_data = test_data_reshape

validation_data_tensor = tf.convert_to_tensor(validation_data.reshape(-1,frames,10,6))
pred = train_model.predict(validation_data_tensor)
print(pred.shape)

17/17 [==============================] - 11s 27ms/step
(529, 24, 10, 6)


# Generate unstd keypoints:

In [11]:
# save num_batches for each video:
num_batch_video = []
num_full_batches_all = 0
for t,x in enumerate(kp_list_test_std):
    if x.shape[0] >= frames:
        num_full_batches = x.shape[0] // frames
        num_full_batches_all += num_full_batches
        num_batch_video.append(num_full_batches)
print(f'number of batches of each video:', len(num_batch_video))

number of batches of each video: 44


In [12]:
# first half of frames: groundtruth; last half of frames: predicted
test_gt_pred = np.concatenate((test_data_reshape[:,:input_frames], pred.reshape(-1,frames,60)[:,input_frames:]), axis = 1)
test_gt_pred.shape

(529, 24, 60)

In [13]:
# unstd for each video:
test_video_unstd_list = []
for video_idx in range(len(num_batch_video)):
    test_video = test_gt_pred[sum(num_batch_video[:video_idx]):sum(num_batch_video[:video_idx+1])]
    test_video_unstd = test_video * range_list[video_idx] + min_list[video_idx]
    test_video_unstd_list.append(test_video_unstd) # unstd video keypoints

# Optical flow and generator

In [14]:
# ####### call the config functions and inference dataloader #########
# config="/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/config/abs-vox.yml"

# # Test dataset
# with open(config) as f:
#     config = yaml.safe_load(f)
# dataset = FramesDataset(is_train=(False), **config['dataset_params'],mode="RNN") # test

# print(len(dataset))
# dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

# # Add this before loading the checkpoint
# torch.cuda.empty_cache()

# ### call the functions        
# generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
#                                         **config['model_params']['common_params'])
# kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
#                             **config['model_params']['common_params'])

# # log_dir="./log/test-reconstruction-vox"
# log_dir="/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/log/test-reconstruction-vox"
# checkpoint="/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/Training_Prediction/FOMM/Trained_Models/vox-cpk.pth"

# if checkpoint is not None:
#     Logger.load_cpk(checkpoint, generator=generator, kp_detector=kp_detector)
# else:
#     raise AttributeError("Checkpoint should be specified for mode='reconstruction'.")
    
# def save_obj(obj, name ):
#     with open('./'+ name + '.pkl', 'wb') as f:
#         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# def load_obj(name ):
#     with open('./' + name + '.pkl', 'rb') as f:
#         return pickle.load(f)

# png_dir = os.path.join(log_dir, 'prediction/png')
# log_dir = os.path.join(log_dir, 'prediction')
  
# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)

# if not os.path.exists(png_dir):
#     os.makedirs(png_dir)

# if torch.cuda.is_available():
#     generator = DataParallelWithCallback(generator)
#     kp_detector = DataParallelWithCallback(kp_detector)

# generator.eval()
# kp_detector.eval()

# prediction_params = config['prediction_params']

# num_epochs = prediction_params['num_epochs']
# lr = prediction_params['lr']
# bs = prediction_params['batch_size']
# num_frames = prediction_params['num_frames']
# loss_list_total = []
# fvd_list_total = []

In [16]:
# Set memory allocation configuration to avoid fragmentation
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'


config_path = "/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/config/abs-vox.yml"
with open(config_path) as f:
    config = yaml.safe_load(f)
    
    # Test dataset
dataset = FramesDataset(is_train=False, **config['dataset_params'], mode="RNN")
print(f"Dataset size: {len(dataset)}")
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)
    
# First create models on CPU, then move to GPU if needed
device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
# Initialize generator and keypoint detector
generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                             **config['model_params']['common_params'])
    
    # Set up logging directories
log_dir = "/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/log/test-reconstruction-vox"
checkpoint_path = "/home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/Training_Prediction/FOMM/Trained_Models/vox-cpk.pth"
    
png_dir = os.path.join(log_dir, 'prediction/png')
log_dir = os.path.join(log_dir, 'prediction')
    
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(png_dir):
    os.makedirs(png_dir)
    
    # Clear GPU cache before loading models
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
    # Custom checkpoint loading with CPU map location
if checkpoint_path is not None:
    print(f"Loading checkpoint from {checkpoint_path}")
        # Load checkpoint to CPU first
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
        
        # Load state dictionaries from checkpoint
    if 'generator' in checkpoint:
        generator.load_state_dict(checkpoint['generator'])
        print("Generator state loaded successfully")
    else:
        print("Warning: Generator state not found in checkpoint")
            
    if 'kp_detector' in checkpoint:
        kp_detector.load_state_dict(checkpoint['kp_detector'])
        print("Keypoint detector state loaded successfully")
    else:
        print("Warning: Keypoint detector state not found in checkpoint")
else:
    raise AttributeError("Checkpoint should be specified for mode='reconstruction'.")
    
    # Move models to device (GPU if available) after loading
generator = generator.to(device)
kp_detector = kp_detector.to(device)
    
    # If GPU is available, wrap models with DataParallel
if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    generator = DataParallelWithCallback(generator)
    kp_detector = DataParallelWithCallback(kp_detector)
    
    # Set models to evaluation mode
generator.eval()
kp_detector.eval()
    
    # Get prediction parameters
prediction_params = config['prediction_params']
num_epochs = prediction_params['num_epochs']
lr = prediction_params['lr']
bs = prediction_params['batch_size']
num_frames = prediction_params['num_frames']
loss_list_total = []
fvd_list_total = []
    
    # Print GPU memory information if available
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"Available GPU memory: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB reserved, {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB allocated")
    
print("Setup complete. Ready for prediction/inference.")

Use predefined train-test split.
using videos from test directory
['id10280#NXjT3732Ekg#001093#001192.mp4', 'id10281#NHARUN9OhSo#000605#000886.mp4', '.ipynb_checkpoints']
Dataset size: 3


/opt/conda/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading checkpoint from /home/jovyan/video-storage/amit_files/Motion_Transfer_Keypoints_Prediction/Keypoints_Prediction/Training_Prediction/FOMM/Trained_Models/vox-cpk.pth
Generator state loaded successfully
Keypoint detector state loaded successfully
GPU: NVIDIA GeForce GTX 1080 Ti
Total GPU memory: 10.90 GB
Available GPU memory: 0.24 GB reserved, 0.23 GB allocated
Setup complete. Ready for prediction/inference.


In [17]:
# Option 1: Install the FFMPEG backend (recommended for most cases)
!pip install "imageio[ffmpeg]"

# Option 2: Install the PyAV backend (alternative)
# !pip install "imageio[pyav]"

  Obtaining dependency information for imageio-ffmpeg from https://files.pythonhosted.org/packages/a0/2d/43c8522a2038e9d0e7dbdf3a61195ecc31ca576fb1527a528c877e87d973/imageio_ffmpeg-0.6.0-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 1.6 MB/s eta 0:00:0000:0100:01


# Original Code 

In [ ]:
# #########  FOMM+VRNN ########

# for it, x in tqdm(enumerate(dataloader)):
#         if config['reconstruction_params']['num_videos'] is not None:
#             if it > config['reconstruction_params']['num_videos']:
#                 break
#         with torch.no_grad():
#             predictions = []
#             visualizations = []

#             ######## keypoints ########
#             kp_driving_video = test_video_unstd_list[it].reshape(-1,10,6)
#             kp_driving_video = torch.tensor(kp_driving_video)
#             kp_source = {"value":kp_driving_video[0,:,:2].reshape(1,10,2),"jacobian":kp_driving_video[0,:,2:].reshape(1,10,2,2)} # kp of the ith frame      
        
#         ##### Start generator
#         loss_list = []
#         fvd_list = []
#         for i in range(((x['video'].shape[2])//frames)*frames): # cut the last <24 frames
#             source = x['video'][:, :, 0]
#             driving = x['video'][:, :, i]
#             kp_driving = {"value":kp_driving_video[i,:,:2],"jacobian":kp_driving_video[i,:,2:]} # kp of the ith frame
#             kp_driving['value'] = kp_driving['value'].reshape(1,10,2)
#             kp_driving['jacobian'] = kp_driving['jacobian'].reshape(1,10,2,2)
#             out = generator(source, kp_source=kp_source, kp_driving=kp_driving)
#             out['kp_source'] = kp_source
#             out['kp_driving'] = kp_driving
#             del out['sparse_deformed']
#             predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])

#             visualization = Visualizer(**config['visualizer_params']).visualize(source=source,
#                                                                                     driving=driving, out=out)
#             visualizations.append(visualization)
#             # mse loss
#             if np.abs(out['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean() != 0:
#                 loss_list.append(np.abs(out['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean())
#                 # Calculate FVD for each frame using ground truth and predicted videos
#                 ground_truth_features = driving.detach().cpu().permute(0,2,3,1).reshape(256,256,3)
#                 predicted_features = out['prediction'].detach().cpu().permute(0,2,3,1).reshape(256,256,3)
#                 fvd_list.append(compute_fvd(ground_truth_features, predicted_features))

#         print("Reconstruction loss: %s" % np.mean(loss_list))
#         loss_list_total.append(np.mean(loss_list))

#         print("FVD Score: %s" % np.mean(fvd_list))
#         fvd_list_total.append(np.mean(fvd_list))

#         predictions = np.concatenate(predictions, axis=1)
#         imageio.imsave(os.path.join(png_dir, x['name'][0] + '.png'), (255 * predictions).astype(np.uint8))
#         image_name = x['name'][0] + config['reconstruction_params']['format']
#         imageio.mimsave(os.path.join(log_dir, image_name), visualizations)

# print("mean Reconstruction loss: %s" % np.mean(loss_list_total)) 
# print("mean FVD score: %s" % np.mean(fvd_list_total)) 

<!-- # -------------------------------------------------------------------------------------------------------------- -->

In [ ]:
#########  FOMM+VRNN ########
import torch
import numpy as np
import os
import imageio
from tqdm import tqdm

# Clear CUDA cache before starting
torch.cuda.empty_cache()

# Monitor memory usage
def print_memory_usage():
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
        print(f"GPU memory reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB")

print("Initial GPU state:")
print_memory_usage()

loss_list_total = []
fvd_list_total = []

# Process batch size - adjust based on your GPU memory
BATCH_SIZE = 2  # Process this many frames at a time

for it, x in tqdm(enumerate(dataloader)):
    if config['reconstruction_params']['num_videos'] is not None:
        if it > config['reconstruction_params']['num_videos']:
            break
            
    print(f"\nProcessing video {it+1}")
    
    with torch.no_grad():
        # These lists store frame paths instead of actual frames to save memory
        prediction_paths = []
        visualizations = []
        
        ######## keypoints ########
        # Convert to half precision to save memory
        kp_driving_video = test_video_unstd_list[it].reshape(-1, 10, 6)
        kp_driving_video = torch.tensor(kp_driving_video, dtype=torch.float16)
        
        # Source keypoints
        kp_source = {
            "value": kp_driving_video[0, :, :2].reshape(1, 10, 2).float(),  # Keep source as float32
            "jacobian": kp_driving_video[0, :, 2:].reshape(1, 10, 2, 2).float()
        }
        
        # Move source keypoint to GPU once
        if torch.cuda.is_available():
            kp_source["value"] = kp_source["value"].cuda()
            kp_source["jacobian"] = kp_source["jacobian"].cuda()
    
    # Load source frame once and keep on GPU
    source = x['video'][:, :, 0]
    if torch.cuda.is_available():
        source = source.cuda()
    
    ##### Start generator
    loss_list = []
    fvd_list = []
    
    # Calculate total frames to process
    total_frames = ((x['video'].shape[2]) // frames) * frames
    
    # Create temp directory for frame storage
    temp_dir = os.path.join(png_dir, f"temp_{x['name'][0]}")
    os.makedirs(temp_dir, exist_ok=True)
    
    # Process in small batches to avoid OOM
    for batch_start in range(0, total_frames, BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE, total_frames)
        
        print(f"Processing frames {batch_start} to {batch_end-1}")
        print_memory_usage()
        
        for i in range(batch_start, batch_end):
            with torch.no_grad():  # Ensure we're not tracking gradients
                # Load driving frame and move to GPU
                driving = x['video'][:, :, i]
                if torch.cuda.is_available():
                    driving = driving.cuda()
                
                # Process keypoints for this frame
                kp_driving = {
                    "value": kp_driving_video[i, :, :2].reshape(1, 10, 2).float(),  # Convert to float32 for computation
                    "jacobian": kp_driving_video[i, :, 2:].reshape(1, 10, 2, 2).float()
                }
                
                # Move to GPU
                if torch.cuda.is_available():
                    kp_driving["value"] = kp_driving["value"].cuda()
                    kp_driving["jacobian"] = kp_driving["jacobian"].cuda()
                
                # Generate frame
                out = generator(source, kp_source=kp_source, kp_driving=kp_driving)
                
                # Add keypoints to output
                out['kp_source'] = kp_source
                out['kp_driving'] = kp_driving
                
                # Remove unnecessary large tensors
                if 'sparse_deformed' in out:
                    del out['sparse_deformed']
                
                # Save prediction directly to file instead of keeping in memory
                frame_pred = np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0]
                frame_path = os.path.join(temp_dir, f"frame_{i:04d}.png")
                imageio.imsave(frame_path, (255 * frame_pred).astype(np.uint8))
                prediction_paths.append(frame_path)
                
                # Create and save visualization
                visualization = Visualizer(**config['visualizer_params']).visualize(
                    source=source.cpu(),
                    driving=driving.cpu(), 
                    out={k: v.cpu() if isinstance(v, torch.Tensor) else v for k, v in out.items()}
                )
                visualizations.append(visualization)
                
                # Calculate metrics on CPU to save GPU memory
                if np.abs(out['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean() != 0:
                    # MSE loss
                    loss_list.append(np.abs(out['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean())
                    
                    # FVD calculation
                    ground_truth_features = driving.detach().cpu().permute(0, 2, 3, 1).reshape(256, 256, 3)
                    predicted_features = out['prediction'].detach().cpu().permute(0, 2, 3, 1).reshape(256, 256, 3)
                    fvd_list.append(compute_fvd(ground_truth_features, predicted_features))
                
                # Explicit cleanup after each frame
                del driving, kp_driving, out, frame_pred
                
            # Clear cache after each frame
            torch.cuda.empty_cache()
            
        # Clear cache after batch
        torch.cuda.empty_cache()
        print("After batch processing:")
        print_memory_usage()
    
    # Print metrics for this video
    print("Reconstruction loss: %s" % np.mean(loss_list))
    loss_list_total.append(np.mean(loss_list))
    
    print("FVD Score: %s" % np.mean(fvd_list))
    fvd_list_total.append(np.mean(fvd_list))
    
    # Combine predictions into a single image using PIL to save memory
    from PIL import Image
    import numpy as np
    
    # Combine all frames into one horizontal image
    images = [Image.open(path) for path in prediction_paths]
    widths, heights = zip(*(i.size for i in images))
    
    total_width = sum(widths)
    max_height = max(heights)
    
    # Create new image with combined dimensions
    new_image = Image.new('RGB', (total_width, max_height))
    
    # Paste images side by side
    x_offset = 0
    for img in images:
        new_image.paste(img, (x_offset, 0))
        x_offset += img.size[0]
    
    # Save combined image
    combined_path = os.path.join(png_dir, x['name'][0] + '.png')
    new_image.save(combined_path)
    
    # Save animation
    image_name = x['name'][0] + config['reconstruction_params']['format']
    imageio.mimsave(os.path.join(log_dir, image_name), visualizations)
    
    # Clean up temporary files
    import shutil
    shutil.rmtree(temp_dir)
    
    # Clear all memory after each video
    del source, kp_source, kp_driving_video
    torch.cuda.empty_cache()
    print("After video processing:")
    print_memory_usage()

print("mean Reconstruction loss: %s" % np.mean(loss_list_total)) 
print("mean FVD score: %s" % np.mean(fvd_list_total))

Initial GPU state:
GPU memory allocated: 0.24 GB
GPU memory reserved: 0.26 GB


0it [00:00, ?it/s]


Processing video 1
Processing frames 0 to 1
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB


/opt/conda/lib/python3.11/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 2 to 3
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 4 to 5
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 6 to 7
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 8 to 9
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 10 to 11
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 12 to 13
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.

1it [3:23:00, 12180.23s/it]

After video processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB

Processing video 2
Processing frames 0 to 1
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 2 to 3
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 4 to 5
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 6 to 7
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 8 to 9
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
After batch processing:
GPU memory allocated: 0.25 GB
GPU memory reserved: 0.26 GB
Processing frames 10 to 11
GPU memory allocated: 0.25 GB
GPU m

# Code by Claude to run it on CPU

In [ ]:
# #########  FOMM+VRNN ########
# import torch
# import numpy as np
# import os
# import imageio
# from tqdm import tqdm

# # Clear CUDA cache before starting
# torch.cuda.empty_cache()

# # Set memory allocation strategy
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# # Use mixed precision to reduce memory usage
# from torch.cuda.amp import autocast

# loss_list_total = []
# fvd_list_total = []

# for it, x in tqdm(enumerate(dataloader)):
#     if config['reconstruction_params']['num_videos'] is not None:
#         if it > config['reconstruction_params']['num_videos']:
#             break
            
#     # Process in smaller batches or frames to reduce memory usage
#     predictions = []
#     visualizations = []
    
#     ######## keypoints ########
#     kp_driving_video = test_video_unstd_list[it].reshape(-1, 10, 6)
#     kp_driving_video = torch.tensor(kp_driving_video)
#     kp_source = {
#         "value": kp_driving_video[0, :, :2].reshape(1, 10, 2),
#         "jacobian": kp_driving_video[0, :, 2:].reshape(1, 10, 2, 2)
#     }
    
#     ##### Start generator
#     loss_list = []
#     fvd_list = []
    
#     # Calculate total frames to process
#     total_frames = (x['video'].shape[2] // frames) * frames
    
#     # Process frames in smaller batches to avoid OOM
#     batch_size = 4  # Process 4 frames at a time, adjust as needed
    
#     for batch_start in range(0, total_frames, batch_size):
#         batch_end = min(batch_start + batch_size, total_frames)
        
#         # Clear cache between batches
#         torch.cuda.empty_cache()
        
#         for i in range(batch_start, batch_end):
#             with torch.no_grad():
#                 source = x['video'][:, :, 0]
#                 driving = x['video'][:, :, i]
                
#                 kp_driving = {
#                     "value": kp_driving_video[i, :, :2].reshape(1, 10, 2),
#                     "jacobian": kp_driving_video[i, :, 2:].reshape(1, 10, 2, 2)
#                 }
                
#                 # Use mixed precision only for the generator forward pass
#                 with autocast():
#                     out = generator(source, kp_source=kp_source, kp_driving=kp_driving)
                
#                 # Convert any half precision outputs back to float32 for visualization
#                 out_float32 = {}
#                 for k, v in out.items():
#                     if isinstance(v, torch.Tensor):
#                         out_float32[k] = v.float()  # Convert to float32
#                     else:
#                         out_float32[k] = v
                
#                 out_float32['kp_source'] = kp_source
#                 out_float32['kp_driving'] = kp_driving
                
#                 # Remove unnecessary data to free memory
#                 if 'sparse_deformed' in out_float32:
#                     del out_float32['sparse_deformed']
                
#                 # Store predictions (already using CPU so no precision issues)
#                 pred_numpy = np.transpose(out_float32['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0]
#                 predictions.append(pred_numpy)
                
#                 # Create visualization with float32 tensors
#                 visualization = Visualizer(**config['visualizer_params']).visualize(
#                     source=source,
#                     driving=driving, 
#                     out=out_float32
#                 )
#                 visualizations.append(visualization)
                
#                 # Calculate metrics
#                 if np.abs(out_float32['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean() != 0:
#                     # Calculate loss
#                     current_loss = np.abs(out_float32['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean()
#                     loss_list.append(current_loss)
                    
#                     # Calculate FVD
#                     ground_truth_features = driving.detach().cpu().permute(0, 2, 3, 1).reshape(256, 256, 3)
#                     predicted_features = out_float32['prediction'].detach().cpu().permute(0, 2, 3, 1).reshape(256, 256, 3)
#                     current_fvd = compute_fvd(ground_truth_features, predicted_features)
#                     fvd_list.append(current_fvd)
                
#                 # Explicitly delete tensors to free up memory
#                 del out, out_float32, source, driving
#                 torch.cuda.empty_cache()
    
#     # Print metrics
#     print("Reconstruction loss: %s" % np.mean(loss_list))
#     loss_list_total.append(np.mean(loss_list))
#     print("FVD Score: %s" % np.mean(fvd_list))
#     fvd_list_total.append(np.mean(fvd_list))
    
#     # Save outputs
#     predictions = np.concatenate(predictions, axis=1)
#     imageio.imsave(os.path.join(png_dir, x['name'][0] + '.png'), (255 * predictions).astype(np.uint8))
    
#     image_name = x['name'][0] + config['reconstruction_params']['format']
#     imageio.mimsave(os.path.join(log_dir, image_name), visualizations)

# print("mean Reconstruction loss: %s" % np.mean(loss_list_total)) 
# print("mean FVD score: %s" % np.mean(fvd_list_total))